# Intro to Dynamic Tables

## Overview

This template demonstrates the power of Snowflake Dynamic Tables through a practical e-commerce analytics scenario. You'll learn how to create, monitor, and manage self-refreshing materialized views that automatically update as source data changes, eliminating the need for complex scheduling and manual maintenance.

**What you'll build:**
- Real-time e-commerce analytics with automatic data freshness
- Layered Dynamic Tables with intelligent dependency management  
- Monitoring and management capabilities for operational excellence
- Cost-effective refresh strategies and best practices

**Key Benefits:**
- ✅ Automatic data freshness without complex scheduling
- ✅ Intelligent dependency management between tables
- ✅ Built-in monitoring and management capabilities
- ✅ Cost-effective refresh strategies
- ✅ Seamless data sharing integration

**Business Scenario:** An e-commerce company needs real-time analytics on customer orders and product performance. Traditional batch processing creates data staleness issues, while manual refresh scheduling is error-prone and resource-intensive. Dynamic Tables solve this by automatically maintaining fresh, query-ready data.

## Step 1: Environment Setup

Let's start by setting up our environment with the standard Snowflake learning configuration and creating a unique schema for this demonstration.

In [ ]:
-- Setup standard learning environment
USE ROLE SNOWFLAKE_LEARNING_ROLE;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
USE DATABASE SNOWFLAKE_LEARNING_DB;

-- Create unique schema for this demo
SET schema_name = CONCAT(CURRENT_USER(), '_DYNAMIC_TABLE');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($schema_name);
USE SCHEMA IDENTIFIER($schema_name);

-- Clean up any existing objects from previous runs
DROP DYNAMIC TABLE IF EXISTS DT_HOURLY_ORDERS;
DROP DYNAMIC TABLE IF EXISTS DT_DAILY_SUMMARY;
DROP DYNAMIC TABLE IF EXISTS DT_CUSTOMER_METRICS;
DROP VIEW IF EXISTS PUBLIC_DAILY_METRICS;
DROP TABLE IF EXISTS ORDERS;
DROP TABLE IF EXISTS PRODUCTS;
DROP TABLE IF EXISTS CUSTOMERS;

SELECT 'Environment setup complete. Using schema: ' || CURRENT_SCHEMA() as STATUS;

## Step 2: Create Sample E-commerce Data

We'll generate realistic sample data representing an active e-commerce platform with customers, products, and order transactions over the past 30 days.

In [ ]:
# Generate sample e-commerce data for Dynamic Tables demo
import snowflake.snowpark as snowpark
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, DateType, FloatType
import pandas as pd
import random
from datetime import datetime, timedelta

from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Generate customers data
customers_data = []
for i in range(1, 1001):  # 1000 customers
    customers_data.append({
        'CUSTOMER_ID': i,
        'CUSTOMER_NAME': f'Customer_{i}',
        'EMAIL': f'customer{i}@email.com',
        'SEGMENT': random.choice(['Premium', 'Standard', 'Basic']),
        'REGISTRATION_DATE': datetime(2023, 1, 1) + timedelta(days=random.randint(0, 365))
    })

# Generate products data
products_data = []
categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']
for i in range(1, 101):  # 100 products
    products_data.append({
        'PRODUCT_ID': i,
        'PRODUCT_NAME': f'Product_{i}',
        'CATEGORY': random.choice(categories),
        'PRICE': round(random.uniform(10.0, 500.0), 2)
    })

# Generate orders data (recent 30 days with realistic patterns)
orders_data = []
order_id = 1
base_date = datetime.now() - timedelta(days=30)

for day in range(30):
    current_date = base_date + timedelta(days=day)
    # More orders on weekends, fewer late at night
    daily_orders = random.randint(20, 100) if current_date.weekday() < 5 else random.randint(50, 150)
    
    for _ in range(daily_orders):
        order_time = current_date + timedelta(
            hours=random.randint(6, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59)
        )
        
        orders_data.append({
            'ORDER_ID': order_id,
            'CUSTOMER_ID': random.randint(1, 1000),
            'PRODUCT_ID': random.randint(1, 100),
            'ORDER_DATE': order_time,
            'QUANTITY': random.randint(1, 5),
            'STATUS': random.choice(['Completed', 'Pending', 'Shipped', 'Cancelled'])
        })
        order_id += 1

# Create and populate base tables
customers_df = session.create_dataframe(customers_data)
customers_df.write.save_as_table("CUSTOMERS", mode="overwrite")

products_df = session.create_dataframe(products_data)
products_df.write.save_as_table("PRODUCTS", mode="overwrite")

orders_df = session.create_dataframe(orders_data)
orders_df.write.save_as_table("ORDERS", mode="overwrite")

print(f"📊 Sample data created successfully:")
print(f"   • {len(customers_data)} customers")
print(f"   • {len(products_data)} products") 
print(f"   • {len(orders_data)} orders (last 30 days)")

Let's verify our sample data is ready for Dynamic Tables:

In [ ]:
-- Verify sample data creation
SELECT 'CUSTOMERS' as TABLE_NAME, COUNT(*) as ROW_COUNT FROM CUSTOMERS
UNION ALL
SELECT 'PRODUCTS' as TABLE_NAME, COUNT(*) as ROW_COUNT FROM PRODUCTS  
UNION ALL
SELECT 'ORDERS' as TABLE_NAME, COUNT(*) as ROW_COUNT FROM ORDERS
ORDER BY TABLE_NAME;

## Step 3: Create Your First Dynamic Table

Now let's create our first Dynamic Table that automatically aggregates order data by hour. This table will refresh every hour and maintain real-time hourly analytics without any manual intervention.

**Key Features:**
- `TARGET_LAG = '1 hour'`: Refreshes automatically within 1 hour of source data changes
- Automatic dependency tracking on ORDERS and PRODUCTS tables
- Self-maintaining aggregations and calculations

In [ ]:
-- Create Dynamic Table for hourly order analytics
CREATE OR REPLACE DYNAMIC TABLE DT_HOURLY_ORDERS
TARGET_LAG = '1 hour'
WAREHOUSE = SNOWFLAKE_LEARNING_WH
AS
SELECT 
    DATE_TRUNC('HOUR', o.ORDER_DATE) AS ORDER_HOUR,
    COUNT(*) AS TOTAL_ORDERS,
    SUM(o.QUANTITY * p.PRICE) AS TOTAL_REVENUE,
    COUNT(DISTINCT o.CUSTOMER_ID) AS UNIQUE_CUSTOMERS,
    COUNT(DISTINCT o.PRODUCT_ID) AS UNIQUE_PRODUCTS,
    AVG(o.QUANTITY * p.PRICE) AS AVG_ORDER_VALUE
FROM ORDERS o
JOIN PRODUCTS p ON o.PRODUCT_ID = p.PRODUCT_ID
WHERE o.STATUS = 'Completed'
GROUP BY DATE_TRUNC('HOUR', o.ORDER_DATE);

Let's examine what we just created:

In [ ]:
-- View recent hourly order data
SELECT 
    ORDER_HOUR,
    TOTAL_ORDERS,
    ROUND(TOTAL_REVENUE, 2) as TOTAL_REVENUE,
    UNIQUE_CUSTOMERS,
    ROUND(AVG_ORDER_VALUE, 2) as AVG_ORDER_VALUE
FROM DT_HOURLY_ORDERS 
ORDER BY ORDER_HOUR DESC 
LIMIT 10;

## Step 4: Create Layered Dynamic Tables

One of the most powerful features of Dynamic Tables is automatic dependency management. Let's create a daily summary table that depends on our hourly table. Snowflake will automatically manage the refresh order!

In [ ]:
-- Create layered Dynamic Table - Daily Summary (depends on hourly data)
CREATE OR REPLACE DYNAMIC TABLE DT_DAILY_SUMMARY
TARGET_LAG = '4 hours'
WAREHOUSE = SNOWFLAKE_LEARNING_WH
AS
SELECT 
    DATE(ORDER_HOUR) AS ORDER_DATE,
    SUM(TOTAL_ORDERS) AS DAILY_ORDERS,
    SUM(TOTAL_REVENUE) AS DAILY_REVENUE,
    MAX(UNIQUE_CUSTOMERS) AS PEAK_HOUR_CUSTOMERS,
    AVG(AVG_ORDER_VALUE) AS DAILY_AVG_ORDER_VALUE,
    COUNT(*) AS ACTIVE_HOURS
FROM DT_HOURLY_ORDERS
GROUP BY DATE(ORDER_HOUR);

In [ ]:
-- Create Advanced Dynamic Table - Customer Lifetime Metrics
CREATE OR REPLACE DYNAMIC TABLE DT_CUSTOMER_METRICS
TARGET_LAG = '2 hours'
WAREHOUSE = SNOWFLAKE_LEARNING_WH
AS
SELECT 
    c.CUSTOMER_ID,
    c.CUSTOMER_NAME,
    c.SEGMENT,
    COUNT(DISTINCT o.ORDER_ID) AS TOTAL_ORDERS,
    SUM(o.QUANTITY * p.PRICE) AS LIFETIME_VALUE,
    AVG(o.QUANTITY * p.PRICE) AS AVG_ORDER_VALUE,
    MAX(o.ORDER_DATE) AS LAST_ORDER_DATE,
    MIN(o.ORDER_DATE) AS FIRST_ORDER_DATE,
    DATEDIFF('day', MIN(o.ORDER_DATE), MAX(o.ORDER_DATE)) AS CUSTOMER_LIFESPAN_DAYS
FROM CUSTOMERS c
LEFT JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID AND o.STATUS = 'Completed'
LEFT JOIN PRODUCTS p ON o.PRODUCT_ID = p.PRODUCT_ID
GROUP BY c.CUSTOMER_ID, c.CUSTOMER_NAME, c.SEGMENT;

## Step 5: Dynamic Tables Monitoring & Management

Dynamic Tables provide built-in monitoring and management capabilities. Let's explore how to monitor refresh status, suspend/resume operations, and trigger manual refreshes.

In [ ]:
-- Monitor Dynamic Tables status and configuration
SELECT 
    TABLE_NAME,
    TARGET_LAG,
    IS_SUSPENDED,
    REFRESH_MODE,
    LAST_SUCCESSFUL_REFRESH,
    DATA_TIMESTAMP
FROM INFORMATION_SCHEMA.DYNAMIC_TABLES 
WHERE TABLE_SCHEMA = CURRENT_SCHEMA()
ORDER BY TABLE_NAME;

In [ ]:
-- Demonstrate suspend and resume operations
-- Suspend the hourly orders table
ALTER DYNAMIC TABLE DT_HOURLY_ORDERS SUSPEND;

-- Check suspension status
SELECT TABLE_NAME, IS_SUSPENDED 
FROM INFORMATION_SCHEMA.DYNAMIC_TABLES 
WHERE TABLE_NAME = 'DT_HOURLY_ORDERS';

-- Resume the table
ALTER DYNAMIC TABLE DT_HOURLY_ORDERS RESUME;

-- Verify resumption
SELECT TABLE_NAME, IS_SUSPENDED 
FROM INFORMATION_SCHEMA.DYNAMIC_TABLES 
WHERE TABLE_NAME = 'DT_HOURLY_ORDERS';

In [ ]:
-- Demonstrate manual refresh for immediate updates
ALTER DYNAMIC TABLE DT_CUSTOMER_METRICS REFRESH;

SELECT 'Manual refresh triggered for DT_CUSTOMER_METRICS' as STATUS;

## Step 6: Explore Your Dynamic Tables Data

Now let's explore the automatically maintained data in our Dynamic Tables:

In [ ]:
-- Explore daily business trends
SELECT 
    ORDER_DATE,
    DAILY_ORDERS,
    ROUND(DAILY_REVENUE, 2) as DAILY_REVENUE,
    PEAK_HOUR_CUSTOMERS,
    ROUND(DAILY_AVG_ORDER_VALUE, 2) as DAILY_AVG_ORDER_VALUE,
    ACTIVE_HOURS
FROM DT_DAILY_SUMMARY
ORDER BY ORDER_DATE DESC
LIMIT 10;

In [ ]:
-- Analyze top customers by lifetime value
SELECT 
    CUSTOMER_NAME,
    SEGMENT,
    TOTAL_ORDERS,
    ROUND(LIFETIME_VALUE, 2) as LIFETIME_VALUE,
    ROUND(AVG_ORDER_VALUE, 2) as AVG_ORDER_VALUE,
    CUSTOMER_LIFESPAN_DAYS
FROM DT_CUSTOMER_METRICS 
WHERE LIFETIME_VALUE > 0
ORDER BY LIFETIME_VALUE DESC 
LIMIT 15;

## Step 7: Data Sharing with Dynamic Tables

Dynamic Tables integrate seamlessly with Snowflake's data sharing capabilities. Let's create a view suitable for sharing that automatically reflects Dynamic Table updates:

In [ ]:
-- Create a view suitable for data sharing
CREATE OR REPLACE VIEW PUBLIC_DAILY_METRICS AS
SELECT 
    ORDER_DATE,
    DAILY_ORDERS,
    DAILY_REVENUE,
    DAILY_AVG_ORDER_VALUE,
    CASE 
        WHEN DAILY_REVENUE > 10000 THEN 'High'
        WHEN DAILY_REVENUE > 5000 THEN 'Medium' 
        ELSE 'Low'
    END AS REVENUE_CATEGORY
FROM DT_DAILY_SUMMARY
WHERE ORDER_DATE >= DATEADD('day', -7, CURRENT_DATE());

-- View the shareable data
SELECT * FROM PUBLIC_DAILY_METRICS ORDER BY ORDER_DATE DESC;

## Key Takeaways

Congratulations! You've successfully implemented and explored Dynamic Tables. Here's what you've learned:

### 🎯 Core Benefits
- **Automatic Data Freshness**: No more complex scheduling or manual refresh processes
- **Intelligent Dependency Management**: Snowflake automatically manages refresh order between layered tables
- **Built-in Monitoring**: Full visibility into refresh status, performance, and health
- **Cost Optimization**: Efficient refresh strategies with TARGET_LAG configuration
- **Seamless Integration**: Works perfectly with data sharing, views, and other Snowflake features

### 💡 Best Practices
1. **Set appropriate TARGET_LAG values** based on business requirements (longer = less compute cost)
2. **Use layered approaches** for complex transformations to optimize refresh efficiency
3. **Monitor refresh patterns** and adjust LAG accordingly for optimal cost/performance balance
4. **Leverage automatic dependency management** to avoid redundant refresh operations
5. **Suspend tables during maintenance** or low-usage periods to save costs
6. **Use manual refresh sparingly** - only when immediate updates are critical

### 🚀 Next Steps
- **Production Implementation**: Apply these patterns to your real-world data pipelines
- **Advanced Features**: Explore clustering keys, data retention, and security policies
- **Cost Monitoring**: Set up alerts and dashboards for refresh cost tracking
- **Data Sharing**: Share your Dynamic Tables with external partners for real-time collaboration

## Cleanup

Run this final cell to clean up all resources created during this demo:

In [ ]:
-- Clean up all created resources
DROP VIEW IF EXISTS PUBLIC_DAILY_METRICS;
DROP DYNAMIC TABLE IF EXISTS DT_CUSTOMER_METRICS;
DROP DYNAMIC TABLE IF EXISTS DT_DAILY_SUMMARY;
DROP DYNAMIC TABLE IF EXISTS DT_HOURLY_ORDERS;
DROP TABLE IF EXISTS ORDERS;
DROP TABLE IF EXISTS PRODUCTS;
DROP TABLE IF EXISTS CUSTOMERS;

SELECT 'All resources cleaned up successfully!' as CLEANUP_STATUS;

## Additional Resources

Ready to dive deeper? Explore these resources:

- 📖 [Snowflake Dynamic Tables Documentation](https://docs.snowflake.com/en/user-guide/dynamic-tables-about)
- 🎯 [Dynamic Tables Best Practices](https://docs.snowflake.com/en/user-guide/dynamic-tables-best-practices)  
- 📊 [Monitoring Dynamic Tables](https://docs.snowflake.com/en/user-guide/dynamic-tables-monitoring)
- 🔗 [Templates Hub](https://app.snowflake.com/templates)

**Happy building with Dynamic Tables! 🚀**